In [1]:
from google.colab import files
files.upload()  # اختر ملف kaggle.json من جهازك

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rahafosama","key":"c01c5bd5c9c7f141d7674b96edb3eb84"}'}

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d puneet6060/intel-image-classification

Dataset URL: https://www.kaggle.com/datasets/puneet6060/intel-image-classification
License(s): copyright-authors
100% 346M/346M [00:16<00:00, 22.6MB/s]
100% 346M/346M [00:16<00:00, 22.0MB/s]


In [4]:
import zipfile
with zipfile.ZipFile("intel-image-classification.zip", 'r') as zip_ref:
    zip_ref.extractall("intel-image-classification")


In [5]:
import os
os.listdir("intel-image-classification")

['seg_train', 'seg_pred', 'seg_test']

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
import os
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten,Dense,Dropout,BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D

In [7]:
#Defining size
batch_size = 32
img_height = 150
img_width = 150


Loading Data¶


In [8]:
train_folder="intel-image-classification/seg_train/seg_train"
train_ds = tf.keras.utils.image_dataset_from_directory(
  train_folder,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 14034 files belonging to 6 classes.


In [9]:
valid_folder="intel-image-classification/seg_test/seg_test"
val_ds = tf.keras.utils.image_dataset_from_directory(
  valid_folder,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 3000 files belonging to 6 classes.


In [10]:
#Labels
label_to_class_name = dict(zip(range(len(train_ds.class_names)), train_ds.class_names))
label_to_class_name

{0: 'buildings',
 1: 'forest',
 2: 'glacier',
 3: 'mountain',
 4: 'sea',
 5: 'street'}

In [11]:
#Size
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

(32, 150, 150, 3)
(32,)




Data preprocessing


In [12]:
train_ds=train_ds.map(lambda x,y:(x/255,y))
val_ds=val_ds.map(lambda x,y:(x/255,y))

In [13]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)


#Training Models¶¶




#VGG16 Model


In [14]:
VGG_model = Sequential()

pretrained_model= tf.keras.applications.VGG16(include_top=False,
                   input_shape=(150,150,3),
                   pooling='max',classes=6,
                   weights='imagenet')
VGG_model.add(pretrained_model)
VGG_model.add(Flatten())
VGG_model.add(Dense(512, activation='relu'))
VGG_model.add(BatchNormalization())  # Batch Normalization layer
VGG_model.add(Dropout(0.5))
VGG_model.add(Dense(6, activation='softmax'))
pretrained_model.trainable=False

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [15]:
VGG_model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [16]:
VGG_model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 512)                 │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 512)                 │         262,656 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 14,982,470 (57.15 MB)

 Trainable params: 266,758 (1.02 MB)

 Non-trainable params: 14,715,712 (56.14 MB)

In [17]:
epochs=15
history2 = VGG_model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 62s 118ms/step - accuracy: 0.7425 - loss: 0.7604 - val_accuracy: 0.8260 - val_loss: 0.4633
Epoch 2/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 39s 89ms/step - accuracy: 0.8369 - loss: 0.4412 - val_accuracy: 0.8497 - val_loss: 0.4029
Epoch 3/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 37s 80ms/step - accuracy: 0.8485 - loss: 0.4048 - val_accuracy: 0.8540 - val_loss: 0.3890
Epoch 4/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8562 - loss: 0.3881 - val_accuracy: 0.8507 - val_loss: 0.3965
Epoch 5/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8649 - loss: 0.3728 - val_accuracy: 0.8463 - val_loss: 0.4117
Epoch 6/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8658 - loss: 0.3716 - val_accuracy: 0.8523 - val_loss: 0.3984
Epoch 7/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 41s 80ms/step - accuracy: 0.8674 - loss: 0.3594 - val_accuracy: 0.8470 - val_loss: 0.4151
Epoch 8/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 35s 80ms/step - accuracy: 0.8701 - loss: 0.3553 -

In [18]:
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 30s 68ms/step - accuracy: 0.8923 - loss: 0.2915
Total Loss : 0.2847
Total Accuracy : 0.8940


In [19]:
ModelLoss, ModelAccuracy = VGG_model.evaluate(val_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.8421 - loss: 0.4087
Total Loss : 0.4008
Total Accuracy : 0.8517


In [20]:
ModelLoss, ModelAccuracy = VGG_model.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy  {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 29s 66ms/step - accuracy: 0.8923 - loss: 0.2915
Total Loss : 0.2847
Total Accuracy  0.8940


In [21]:
ModelLoss, ModelAccuracy = VGG_model.evaluate(val_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy  {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 65ms/step - accuracy: 0.8421 - loss: 0.4087
Total Loss : 0.4008
Total Accuracy  0.8517




#MobileNet Model


In [23]:
mobile_net = Sequential()
pretrained_model= tf.keras.applications.MobileNetV2(include_top=False,
                   input_shape=(150,150,3),
                   pooling='max',classes=6,
                   weights='imagenet')
mobile_net.add(pretrained_model)
mobile_net.add(Flatten())
mobile_net.add(Dense(512, activation='relu'))
mobile_net.add(BatchNormalization())  # Batch Normalization layer
mobile_net.add(Dropout(0.5))
mobile_net.add(Dense(6, activation='softmax'))
pretrained_model.trainable=False

<ipython-input-23-4a2e8e1ff51e>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  pretrained_model= tf.keras.applications.MobileNetV2(include_top=False,


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [24]:
mobile_net.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

In [25]:
mobile_net.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 1280)                │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 1280)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 512)                 │           2,048 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │           3,078 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,918,982 (11.14 MB)

 Trainable params: 659,974 (2.52 MB)

 Non-trainable params: 2,259,008 (8.62 MB)

In [26]:
epochs=15
history3 = mobile_net.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 48s 82ms/step - accuracy: 0.8178 - loss: 0.5623 - val_accuracy: 0.8933 - val_loss: 0.3011
Epoch 2/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.8923 - loss: 0.2995 - val_accuracy: 0.8970 - val_loss: 0.2839
Epoch 3/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9044 - loss: 0.2663 - val_accuracy: 0.8913 - val_loss: 0.2985
Epoch 4/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 22ms/step - accuracy: 0.9099 - loss: 0.2485 - val_accuracy: 0.8820 - val_loss: 0.3163
Epoch 5/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 21ms/step - accuracy: 0.9157 - loss: 0.2422 - val_accuracy: 0.8867 - val_loss: 0.3022
Epoch 6/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 17ms/step - accuracy: 0.9180 - loss: 0.2251 - val_accuracy: 0.8887 - val_loss: 0.3101
Epoch 7/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9178 - loss: 0.2176 - val_accuracy: 0.8850 - val_loss: 0.3260
Epoch 8/15
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9262 - loss: 0.2082 - val_ac

In [34]:

ModelLoss, ModelAccuracy = mobile_net.evaluate( train_ds)

print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


439/439 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.9518 - loss: 0.1258
Total Loss : 0.1190
Total Accuracy : 0.9543


In [33]:

ModelLoss, ModelAccuracy = mobile_net.evaluate( val_ds)


print('Total Loss : {:.4f}'.format(ModelLoss))
print('Total Accuracy : {:.4f}'.format(ModelAccuracy))


94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.8872 - loss: 0.3990
Total Loss : 0.4018
Total Accuracy : 0.8857


In [29]:
epochs=25
history3 = mobile_net.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

Epoch 1/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9349 - loss: 0.1751 - val_accuracy: 0.8667 - val_loss: 0.4046
Epoch 2/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 10s 17ms/step - accuracy: 0.9343 - loss: 0.1751 - val_accuracy: 0.8753 - val_loss: 0.3932
Epoch 3/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9365 - loss: 0.1679 - val_accuracy: 0.8757 - val_loss: 0.4037
Epoch 4/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9403 - loss: 0.1610 - val_accuracy: 0.8817 - val_loss: 0.3726
Epoch 5/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 17ms/step - accuracy: 0.9401 - loss: 0.1620 - val_accuracy: 0.8783 - val_loss: 0.3841
Epoch 6/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.9364 - loss: 0.1706 - val_accuracy: 0.8740 - val_loss: 0.3989
Epoch 7/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9406 - loss: 0.1552 - val_accuracy: 0.8683 - val_loss: 0.4334
Epoch 8/25
439/439 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 0.9395 - loss: 0.1548 - val_a

439/439 ━━━━━━━━━━━━━━━━━━━━ 28s 65ms/step - accuracy: 0.8923 - loss: 0.2915
Total Loss : 0.2847
Total Accuracy : 0.8940


In [39]:
import glob  as gb
# تعريف مسار بيانات الاختبار
testpath = 'intel-image-classification/'  # المسار الرئيسي

# إعداد بيانات الاختبار
s = 100  # حجم الصورة المطلوب
X_test = []
y_test = []

for folder in os.listdir(testpath + 'seg_test/'):
    files = gb.glob(pathname=str(testpath + 'seg_test//' + folder + '/*.jpg'))
    for file in files:
        image = cv2.imread(file)
        image_array = cv2.resize(image, (s, s))  # إعادة تغيير الحجم
        X_test.append(image_array)
        y_test.append(code[folder])  # ربط الصورة بالفئة الخاصة بها

X_test = np.array(X_test)  # تحويل إلى مصفوفة NumPy
y_test = np.array(y_test)  # تحويل إلى مصفوفة NumPy

print(f'X_test shape: {X_test.shape}')
print(f'y_test shape: {y_test.shape}')

X_test shape: (0,)
y_test shape: (0,)
